## Use this starter notebook to run some new experiments

In [ ]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/GresvanAIE.xz"

In [ ]:
# load cell magic things
%load_ext line_profiler
%load_ext Cython

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_data import Race
from sc2.unit import Unit
from sc2.units import Units

from tests.load_bot_from_pickle import get_map_specific_bot

In [ ]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

In [ ]:
from sc2.constants import CREATION_ABILITY_FIX
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from collections import Counter
from sc2.data import Race
from sc2.unit import Unit

def _abilities_count_and_build_progress(self):
    """Cache for the already_pending function, includes protoss units warping in,
    all units in production and all structures, and all morphs"""
    abilities_amount: Counter[AbilityId] = Counter()
    unit: Unit
    for unit in self.units + self.structures:
        for order in unit.orders:
            abilities_amount[order.ability.exact_id] += 1
        if not unit.is_ready and (self.race != Race.Terran or not unit.is_structure):
            # If an SCV is constructing a building, already_pending would count this structure twice
            # (once from the SCV order, and once from "not structure.is_ready")
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    # Hotfix for archons in morph state
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    abilities_amount[creation_ability] += 2
                else:
                    # Hotfix for rich geysirs
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    abilities_amount[creation_ability] += 1
            else:
                creation_ability: AbilityId = self.game_data.units[unit.type_id.value].creation_ability.exact_id
                abilities_amount[creation_ability] += 1

    return abilities_amount

In [ ]:
%timeit _abilities_count_and_build_progress(bot)

In [17]:
%%cython
from libc.stdlib cimport malloc, free
from libc.string cimport memset
from cython cimport boundscheck, wraparound
from sc2.data import Race
from cython_extensions.ability_mapping_notebook import map_value
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId


cdef int STRUCT_ABILITIES[1620]  # raw C array

memset(STRUCT_ABILITIES, 0, 1620 * sizeof(int))
STRUCT_ABILITIES[AbilityId.BUILD_REACTOR_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.BUILD_TECHLAB_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.TERRANBUILD_COMMANDCENTER.value] = 2
STRUCT_ABILITIES[AbilityId.ZERGBUILD_HATCHERY.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOLAIR_LAIR.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOHIVE_HIVE.value] = 1




cdef struct AbilityCount:
    int ability_id
    int count



@boundscheck(False)
@wraparound(False)
cpdef AbilityCount[:] abilities_count_structures1(object bot):
    cdef int MAX_ABILITIES = 2200
    cdef AbilityCount* arr = <AbilityCount*> malloc(MAX_ABILITIES * sizeof(AbilityCount))
    memset(arr, 0, MAX_ABILITIES * sizeof(AbilityCount))

    cdef object unit, order
    cdef int aid
    cdef object structures = bot.structures
    cdef object workers = bot.workers

    # Workers orders → ability_id count
    for unit in workers:
        for order in unit.orders:
            aid = <int> order.ability._proto.ability_id
            if 0 <= aid < MAX_ABILITIES:
                arr[aid].count += 1

    # Structures → build progress < 1.0 → increment creation ability
    race = bot.race
    if race == Race.Protoss:
        for unit in structures:
            if <double> unit.build_progress < 1.0:
                aid = <int> map_value(unit.type_id.value)
                if aid != -1:
                    arr[aid].count += 1
    elif race == Race.Zerg:
        for unit in structures:
            aid = <int> map_value(unit.type_id.value)
            if <double> unit.build_progress < 1.0 and aid != -1:
                arr[aid].count += 1
            elif STRUCT_ABILITIES[aid] == 2:
                for order in unit.orders:
                    aid = <int> order.ability._proto.ability_id
                    if STRUCT_ABILITIES[aid]:
                        arr[aid].count += 1
    else:  # Terran
        for unit in structures:
            aid = <int> map_value(unit.type_id.value)
            if <double> unit.build_progress < 1.0 and STRUCT_ABILITIES[aid]:
                arr[aid].count += 1
            elif STRUCT_ABILITIES[aid] == 2:
                for order in unit.orders:
                    aid = <int> order.ability._proto.ability_id
                    if STRUCT_ABILITIES[aid]:
                        arr[aid].count += 1

    return <AbilityCount[:MAX_ABILITIES]> arr


In [18]:
%timeit abilities_count_structures1(bot)

22.5 μs ± 2.36 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%%cython
from cython_extensions.ability_order_tracker import abilities_count_structures


Content of stdout:
_cython_magic_5846358c6f4afc72b872fc3f083ab9c5209b9a4c3080e709d648e8a594c60eb0.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_5846358c6f4afc72b872fc3f083ab9c5209b9a4c3080e709d648e8a594c60eb0.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_5846358c6f4afc72b872fc3f083ab9c5209b9a4c3080e709d648e8a594c60eb0.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

In [16]:
%timeit abilities_count_structures(bot)

13.7 μs ± 1.27 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
from cython_extensions import cy_structure_pending


In [ ]:
%timeit -n 10000 cy_structure_pending(bot, UnitTypeId.BARRACKS)

In [ ]:
%timeit -n 10000 bot.already_pending(UnitTypeId.BARRACKS)

In [ ]:
%%cython
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping cimport map_value
from cython_extensions.ability_order_tracker import cy_abilities_count_structures

cdef struct AbilityCount:
    int ability_id
    int count

@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending_func(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        object counts_and_progress
        object creation_ability = None
        object local_get
        int ability_int
        object ability_obj
        int count
        int target = <int> structure_type.value
        cdef AbilityCount item

    # Use optimized Cython function to get ability counts

    counts_and_progress = cy_abilities_count_structures(bot) #returns a c array memoryview


    # local_get reduces repeated attribute accesses
    arr_len = counts_and_progress.shape[0]
    target_created_ability = <int> map_value(target)
    #print("target_created_ability:", target_created_ability, "struct:",  structure_type)
    if 0 <= target_created_ability < arr_len:
        item = counts_and_progress[target_created_ability]
        num_pending += item.count
    return num_pending

In [ ]:
%timeit cy_structure_pending_func(bot, UnitTypeId.BARRACKS)

In [ ]:
worker=bot.workers.first

In [ ]:
%timeit worker.orders[0].ability.exact_id

In [ ]:
%timeit worker.orders[0].ability._proto.ability_id